In [91]:
import numpy as np
import chemicals
from chemicals import *

import cobra
cobra_config = cobra.Configuration()

from cobra import Model, Reaction, Metabolite
import cobra.test
import os
from os.path import join

data_dir = cobra.test.data_dir



In [78]:
 #build from plasson string test
class MSEquation:

    def __init__(self, stoichiometry, d):
        self.equation = stoichiometry
        self.direction = d


#ASSUMPTIONS:
#user has at least one space before metabolites and arrow (<=, <=>, or =>)

#NOTE: more spaces are better

def build_from_palsson_string(equation_string, default_group = 'c'): #add default group

    def clean_ends(lst):
        for i in range(len(lst)):
            # remove whitespace from the front
            while (lst[i][0] == ' '):
                lst[i] = lst[i][1:]
            # remove whitespace from the back
            while (lst[i][-1] == ' '):
                lst[i] = lst[i][:-1]
        return lst

    def get_coef_and_group(lst, return_dict, side):  #for side variable, -1 is left side, 1 is right side, for coeficients
        for reactant in lst:
            coeficient = side
            if reactant.find('(') != -1 or reactant.find(')') != -1:   # if one is present, check to make sure both there
                if equation_string.find(')') == -1:
                    raise Exception("Error, ')' character missing in string", reactant)
                if equation_string.find('(') == -1:
                    raise Exception("Error, '(' character missing in string", reactant)
                number = ''
                position = 1
                while reactant[position] != ')':  
                    number += reactant[position]
                    position += 1
                coeficient = side * int(number)
                reactant = reactant[position + 1:]

            identifier = default_group
            if reactant.find('[') != -1 or reactant.find(']') != -1:  # if one is present, check to make sure both there
                #check to see both are present
                if equation_string.find(']') == -1:
                    raise Exception("Error, ']' character missing in string", reactant)
                if equation_string.find('[') == -1:
                    raise Exception("Error, '[' character missing in string", reactant)              
                s = ''
                position = -2
                while reactant[position] != '[':   
                    s = reactant[position] + s
                    position -= 1
                identifier = s
                reactant = reactant[:position]

            return_dict[(reactant, identifier)] = coeficient
        return return_dict
    
    #check for the '=' character, throw exception otherwise
    if equation_string.find('=') == -1:
        raise Exception("Error, '=' character missing, unable to split string", equation_string)

    #check direction
    reversible = False
    right = False
    left = False
    ret_str = ''
    reversible = equation_string.find('<=>') != -1
    if reversible:
        ret_str = 'reversable'
    else:  # not two ways, so check right
        right = equation_string.find('=>') != -1
        if right:
            ret_str = 'right'
        else:  # if not right, check left
            left = equation_string.find('<=') != -1
            if left: 
                ret_str = 'left'
            else:  # if not left, error
                ret_str = "error - no direction found"


    #cpd00001 + cpd00002[e] => (2)cpd00003 + cpd00004
    # get substrings for either side of the euqation
    reactants_substring_list = equation_string[0:equation_string.find('=') - 1].split('+')
    products_substring_list = equation_string[equation_string.find('=') + 2:len(equation_string)].split('+')

    # clean up our substrings:
    clean_ends(reactants_substring_list)
    clean_ends(products_substring_list)

    variables = {}
    # add reactants to the dictionary

    get_coef_and_group(reactants_substring_list, variables, -1)
    get_coef_and_group(products_substring_list, variables, 1)

    ret_mse = MSEquation(variables, ret_str)

    return ret_mse
    
test = build_from_palsson_string('cpd00001 + cpd00002[e] <= (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] <=> (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] => (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] = (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

#test = build_from_palsson_string('cpd00001 + cpd00002[e]  (2)cpd00003 + cpd00004')# <- working, missing =
#print(test.equation)
#print(test.direction)

#test = build_from_palsson_string('cpd00001 + cpd00002[e => (2)cpd00003 + cpd00004')# <- working, missing ]
#print(test.equation)
#print(test.direction)

#test = build_from_palsson_string('cpd00001 + cpd00002e] => (2)cpd00003 + cpd00004')# <- working, missing [
#print(test.equation)
#print(test.direction)

#test = build_from_palsson_string('cpd00001 + cpd00002[e] => (2cpd00003 + cpd00004')# <- working, missing )
#print(test.equation)
#print(test.direction)

#test = build_from_palsson_string('cpd00001 + cpd00002[e] => 2)cpd00003 + cpd00004')# <- working, missing (
#print(test.equation)
#print(test.direction)

{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
left
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
reversable
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
right
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
error - no direction found


In [82]:
#Add Custom Reaction test

#LIMITATIONS:
#also, no support for gpr or genome

def add_custom_reaction(model,rxn_id,MSEquation,gpr = None,genome = None):
    new_rxn = Reaction(id = rxn_id)
    #going on the assumption that all metabolites are present in the model
    metabolites = {}
    for key in MSEquation.equation:
        met_id = key[0]+'_'+key[1]
        if met_id in model.metabolites:
            metabolites[met_id] = MSEquation.equation[key]
        else:
            raise Exception("Error,", met_id, "not in model metabolites list")
    model.add_reaction(new_rxn)
    #new_rxn.gene_reaction_rule = gpr
    new_rxn.add_metabolites(metabolites)
    
    #adjust the bounds based on the arrow direction  -1000, 1000, 0
    if MSEquation.direction == 'left':
        new_rxn.lower_bound = -1000
        new_rxn.upper_bound = 0
    if MSEquation.direction == 'reversable':
        new_rxn.lower_bound = -1000
        new_rxn.upper_bound = 1000
   
    
test = build_from_palsson_string('octapb + cysi__L[e] => (2)dhap + prbatp')
model = cobra.io.load_json_model("iML1515.json")
add_custom_reaction(model, 'test_id', test)
model.reactions.get_by_id('test_id')

test = build_from_palsson_string('octapb + cysi__L[e] <=> (2)dhap + prbatp')
add_custom_reaction(model, 'test_id2', test)
print(model.reactions.get_by_id('test_id2'))

test = build_from_palsson_string('octapb + cysi__L[e] <= (2)dhap + prbatp')
add_custom_reaction(model, 'test_id3', test)
print(model.reactions.get_by_id('test_id3'))

Reaction identifier,test_id3
Name,
Memory address,0x02dcbbf599a0
Stoichiometry,cysi__L_e + octapb_c <-- 2 dhap_c + prbatp_c L Cystine C6H12N2O4S2 + Octanoate (protein bound) <-- 2 Dihydroxyacetone phosphate + 1-(5-Phosphoribosyl)-ATP
GPR,
Lower bound,-1000
Upper bound,0


In [119]:
#remove reactions test
def remove_reactions(model,rxn_id_list = []):
    for rxn_id in rxn_id_list:
        if not model.reactions.has_id(rxn_id):
            raise Exception('Reaction',rxn_id,'is not in the model.')
        model.remove_reactions([rxn_id])
    
model = cobra.io.load_json_model("iML1515.json")

print(len(model.reactions))
lst = ['XPPT','HXPRT']
remove_reactions(model,lst)
print(len(model.reactions))

lst = ['C']
remove_reactions(model,lst)  #thows warning orrectly

2712
2710


Exception: ('Reaction', 'C', 'is not in the model.')

In [111]:
#edit biomass compound test

#assumptions: 
#compound is already present in the reaction, we are just changing the coeficient

#look into metabolite addition
#add a line to check if id is in model.metabolites, for both biomass and cpd_id
#if no biomass_id, create a new reaction

#ignore rescale for now, we will add a new function
#librarry chemicals can get data for the weights
#new function: compute molecular weight, take in matabolite, compute molecular weight of the metabolite

def edit_biomass_compound(model,biomass_id,cpd_id,new_coef,rescale = 1):
    if biomass_id in model.reactions:
        if cpd_id in model.metabolites:
            model.reactions.get_by_id(biomass_id).add_metabolites({model.metabolites.get_by_id(cpd_id): new_coef}, combine=False)
        else:
            raise Exception('Metabolite',cpd_id,' is not in the model.')
    else: # if there is no biomass reaction
        biomass_rxn = Reaction(biomass_id)
        model.add_reaction(biomass_rxn)
        if cpd_id in model.metabolites:
            biomass_rxn.add_metabolites({model.metabolites.get_by_id(cpd_id): new_coef})
        else:
            raise Exception('Metabolite ',cpd_id,' is not in the model.')
        
    
model = cobra.io.load_json_model("iML1515.json")
print(model.reactions[0])
print(model.reactions[0].metabolites)
print()
edit_biomass_compound(model, 'CYTDK2', 'cmp_c', 2)
print(model.reactions[0].metabolites)


model = cobra.io.load_json_model("iML1515.json")
print(model.reactions[0])
print(model.reactions[0].metabolites)
print()
edit_biomass_compound(model, 'CYTDK', 'cmp_c', 2)  #typo here, seeing what it does to a nonexistant reaction
print(model.reactions[0].metabolites)

# ^ properly throws keyerror


model = cobra.io.load_json_model("iML1515.json")
print(model.reactions[0])
print(model.reactions[0].metabolites)
print()
edit_biomass_compound(model, 'CYTDK2', 'cmp_', 2)  #typo here, seeing what it does to a nonexistant metabolite
print(model.reactions[0].metabolites)

# ^ properly throws keyerror

CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
{<Metabolite cmp_c at 0x2dcc3b3a3d0>: 1.0, <Metabolite cytd_c at 0x2dcc33bfcd0>: -1.0, <Metabolite gdp_c at 0x2dcbfde3580>: 1.0, <Metabolite gtp_c at 0x2dcc3b3a490>: -1.0, <Metabolite h_c at 0x2dcc28a5a00>: 1.0}

{<Metabolite cmp_c at 0x2dcc3b3a3d0>: 2, <Metabolite cytd_c at 0x2dcc33bfcd0>: -1.0, <Metabolite gdp_c at 0x2dcbfde3580>: 1.0, <Metabolite gtp_c at 0x2dcc3b3a490>: -1.0, <Metabolite h_c at 0x2dcc28a5a00>: 1.0}
CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
{<Metabolite cmp_c at 0x2dcb10ad880>: 1.0, <Metabolite cytd_c at 0x2dcc2be91c0>: -1.0, <Metabolite gdp_c at 0x2dcc74d43a0>: 1.0, <Metabolite gtp_c at 0x2dcb10ad940>: -1.0, <Metabolite h_c at 0x2dcb37c7e50>: 1.0}

{<Metabolite cmp_c at 0x2dcb10ad880>: 1.0, <Metabolite cytd_c at 0x2dcc2be91c0>: -1.0, <Metabolite gdp_c at 0x2dcc74d43a0>: 1.0, <Metabolite gtp_c at 0x2dcb10ad940>: -1.0, <Metabolite h_c at 0x2dcb37c7e50>: 1.0}
CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
{<Metabolit

Exception: ('Metabolite', 'cmp_', ' is not in the model.')

In [101]:
def compute_molecular_weight(model, metabolite_id):
    if metabolite_id not in model.metabolites:
        raise Exception('Error, metabolite',metabolite_id,'not found in the model')
    return model.metabolites.get_by_id(metabolite_id).formula_weight
    
model = cobra.io.load_json_model("iML1515.json")
print('formula', model.metabolites.get_by_id('octapb_c').formula)
print(compute_molecular_weight(model, 'octapb_c'))

formula C8H15O
127.2041


In [104]:
#copy model reactions test
#how to do
#create two models
#delete function from 1
#add to the second
#check to see if reaction counts are the same

def copy_model_reactions(model,source_model,rxn_id_list = []):
    for rxnid in rxn_id_list:
        if rxnid in source_model.reactions:
            model.add_reaction(source_model.reactions.get_by_id(rxnid))
        else:
            raise Exception('The reaction', rxnid,'in the reaction list is not in the model.')
        
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
lst = ['CYTDK2']
print('model 1 reaction count',len(model.reactions))
remove_reactions(model,lst)
print('model 1 reaction count with removal',len(model.reactions))
copy_model_reactions(model, model2, lst)
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

#testing on copying a id that already exists in the duplicate
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
copy_model_reactions(model, model2, lst)
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

#testing on copying a id that does not exist in the origional
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
lst = ['C']
copy_model_reactions(model, model2, lst)
print('thows keyerror properly')
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

model 1 reaction count 2712
model 1 reaction count with removal 2711
model 1 reaction count 2712
model 2 reaction count 2712


Ignoring reaction 'CYTDK2' since it already exists.


model 1 reaction count 2712
model 2 reaction count 2712


Exception: ('The reaction', 'C', 'in the reaction list is not in the model.')

In [121]:
#edit reaction progam
#ASSUMPTIONS:
#an arrow will exist in the program, either =>, <=>, or <=
def edit_reaction(model,rxn_id,direction = None,gpr = None,genome = None):
    # Direction: =>, <=, or <=>
    if direction is not None:
        lower_bound = model.reactions.get_by_id(rxn_id).lower_bound
        upper_bound = model.reactions.get_by_id(rxn_id).upper_bound

        if lower_bound < 0 and upper_bound > 0: # rxn_id is reversible
            if direction == "=>":
                model.reactions.get_by_id(rxn_id).lower_bound = 0
            elif direction == "<=":
                model.reactions.get_by_id(rxn_id).upper_bound = 0
        elif lower_bound == 0 and upper_bound > 0: # rxn_id is forward only
            if direction == "<=":
                model.reactions.get_by_id(rxn_id).lower_bound = -1*upper_bound
                model.reactions.get_by_id(rxn_id).upper_bound = 0
            elif direction == "<=>":
                model.reactions.get_by_id(rxn_id).lower_bound = -1*upper_bound
        elif lower_bound < 0 and upper_bound == 0: # rxn_id is reverse only
            if direction == "=>":
                model.reactions.get_by_id(rxn_id).lower_bound = 0
                model.reactions.get_by_id(rxn_id).upper_bound = -1*lower_bound
            elif direction == "<=>":
                model.reactions.get_by_id(rxn_id).upper_bound = -1*lower_bound

    # Specify GPR as a string with boolean conditions (e.g. "(b0001 and b0002) or b1010").
    try:
        if gpr is not None:
            model.reactions.get_by_id(rxn_id).gene_reaction_rule = gpr
    except:
        raise Exception('invalid gpr statement, check parentheses')  #not working, unsure exactly why
        
        
model = cobra.io.read_sbml_model(join(data_dir, "iML1515.xml"))

# Changing a reversible reaction to a forward (irreversible) reaction
# and changing the gpr
rxn = 'ICDHyr'
print('reversibility before change:',model.reactions.get_by_id(rxn).reversibility)
print('forward:',model.reactions.get_by_id(rxn).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn).reverse_variable)
print('gpr:',model.reactions.get_by_id(rxn).gene_name_reaction_rule)
edit_reaction(model, rxn, "=>", "(b0001 and b0002 or b1010")
print('reversibility after change:',model.reactions.get_by_id(rxn).reversibility)
print('forward:',model.reactions.get_by_id(rxn).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn).reverse_variable)
print('gpr:',model.reactions.get_by_id(rxn).gene_name_reaction_rule)

# Changing a forward (irreversible) reaction to a reversible reaction
print(' ')
rxn2 = 'HXPRT'
print('reversibility before change:',model.reactions.get_by_id(rxn2).reversibility)
print('forward:',model.reactions.get_by_id(rxn2).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn2).reverse_variable)
edit_reaction(model, rxn2, "<=>")
print('reversibility after change:',model.reactions.get_by_id(rxn2).reversibility)
print('forward:',model.reactions.get_by_id(rxn2).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn2).reverse_variable)

# Changing a forward (irreversible) reaction to a reverse (irreversible) reaction
print(' ')
rxn3 = 'CYTDK2'
print('reversibility before change:',model.reactions.get_by_id(rxn3).reversibility)
print('forward:',model.reactions.get_by_id(rxn3).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn3).reverse_variable)
edit_reaction(model, rxn3, "<=")
print('reversibility after change:',model.reactions.get_by_id(rxn3).reversibility)
print('forward:',model.reactions.get_by_id(rxn3).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn3).reverse_variable)

#add test with unclear gpr, missing parentheseese, use try, catch

reversibility before change: True
forward: 0 <= ICDHyr <= 1000.0
reverse: 0 <= ICDHyr_reverse_7f84b <= 1000.0
gpr: icd
reversibility after change: False
forward: 0 <= ICDHyr <= 1000.0
reverse: 0 <= ICDHyr_reverse_7f84b <= 0


C:\Users\jason\anaconda3\lib\site-packages\cobra\core\reaction.py:438: UserWarning: malformed gene_reaction_rule '(b0001 and b0002 or b1010' for <Reaction ICDHyr at 0x2dcc5d0e640>
  warn("malformed gene_reaction_rule '%s' for %s" % (new_rule, repr(self)))


SyntaxError: unexpected EOF while parsing (<string>, line 1)